# Api Twitter
* https://datascienceparichay.com/article/get-data-from-twitter-api-in-python-step-by-step-guide/
* https://developer.twitter.com/en/portal/projects/1391772372416532480/apps/20820453/keys

Análisis de sentimiento en Español:
* https://platzi.com/tutoriales/1874-python-lenguaje-natural/5654-realiza-un-analisis-de-sentimiento-en-3-pasos-con-python/
    * Ejemplo https://twitter.com/whaleandjaguar_?lang=en

## Tutorial de búsqueda
### Acceso para búsqueda avanzada
Para acceder como usario ver [aquí](https://www.geeksforgeeks.org/python-api-me-in-tweepy/)

In [1]:
import tweepy as tw
import getpass

In [2]:
my_api_key=getpass.getpass()

 ·························


In [3]:
my_api_secret=getpass.getpass()

 ··················································


In [4]:
auth = tw.OAuthHandler(my_api_key, my_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

In [6]:
search_query = "#SOSColomia OR #ParoNacional OR #Cali OR #UribeTieneLaRazón OR #YoApoyoAlEsmad OR #NoMasParo"

In [7]:
# get tweets from the API
tweets = tw.Cursor(api.search,
              q=search_query,
              lang="es",
              since="2021-03-01" #https://es.wikipedia.org/wiki/Protestas_en_Colombia_de_2021
                  ).items(50) #empty for all 

Cada tweet es un objeto con uun tweet completo desglosado en: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet

In [8]:
# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Total Tweets fetched: 50


El módulo `pickle` permité salvar objetos en archivos binarios. De ésta manera podemos grabar la información completa sobre la búsqueda para un análisis posterior

In [9]:
#Save the list of tweet objects as a pickle file
import pickle
f=open('tweets_copy.pickle','wb')
pickle.dump(tweets_copy,f)
f.close()

In [13]:
!ls -lh tweets_copy.pickle

-rw-r--r-- 1 restrepo restrepo 26M May 22 05:24 tweets_copy.pickle


Cada elemento de la lista es un objeto tweet, que equivala a una fila de una base de datos no relacional. Su documentación está disponible [aquí](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet).

Para acceder al primer hastashg de un tweet debemos caminar la estructura:

lista → diccionario → lista → diccionario

In [18]:
tweets_copy[1].entities['hashtags']#[0]['text']

[]

## Ejemplo de DataFrame con información básica

In [19]:
import pandas as pd

# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    
    original=None
    if tweet._json.get('retweeted_status'):
        original=tweet.retweeted_status.id
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'id':tweet.id,
                                               'original_id':original}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head()

,user_name,user_location,user_description,user_verified,date,text,hashtags,source,id,original_id
0,Alonso,Asia,I ask the international community to help the ...,False,2021-05-22 10:17:25,RT @Blg1M: Cartago #VALLE 21de MAYO2021 🚨🆘 \n#...,"[VALLE, NosEstanMantando, Cartago, ParoNaciona...",Twitter for Android,1396047538369830918,1395961270948536322
1,Sandra Suarez - opiniones personales,,,False,2021-05-22 10:16:48,"RT @CapitnColombia: Por su presente, por su fu...",None,Twitter for Android,1396047383784529920,1395886242731110417
2,@marioperico4,,abajotwitter,False,2021-05-22 10:16:41,RT @martinquinco: @jflafaurie @UNPColombia Des...,"[NoMasParo, NoMasBloqueos, YoApoyoALaFuerzaPub...",Twitter for Android,1396047356123041792,1395755520607334401
3,maravillamil,,"Diseñador, investigador, antiuribista y comuni...",False,2021-05-22 10:16:40,RT @martinquinco: @jflafaurie @UNPColombia Des...,"[MuyParacoSi, UribeParacoHpta]",Twitter for Android,1396047352553689096,1396043260926337024
4,Mauro Gabriel Escalona Quiroz,,"Antiuribista 100% y 24/7, por que Colombia mer...",False,2021-05-22 10:16:36,RT @Contagioradio1: #Bogotá Chicos de la guard...,[Bogotá],Twitter for Android,1396047336439259136,1395937231060230145


El url del tweet se obtiene a partir del 'id' poniendolo al final del siguiente link
https://twitter.com/twitter/status/1394753543010230274. Si el 'text' comienza con RT, el tweet original estarìa en el 'original_id', por ejemplo https://twitter.com/twitter/status/1394277783040368641

In [20]:
tweets_df.shape

(5000, 10)

In [21]:
#Save to standard JSON file
tweets_df.to_json('tweets_df.json',orient='records')

### Tweets with timelines

We can use the `max_id` parameter to load all the tweets: See https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/guides/working-with-timelines

In [38]:
import pandas as pd
tweets_df = pd.DataFrame()
max_items=5000
steps=4
#Get the first tweet
tweets = tw.Cursor(api.search,
              q=search_query,
              lang="es",
              since="2021-03-01"
                  ).items(1) #empty for all
max_id=[t._json.get('id') for t in tweets][0]

#Get max_items*steps tweets
for i in range(steps):
    print(i,end='\r')
    tweets = tw.Cursor(api.search,
                  q=search_query,
                  lang="es",
                  since="2021-03-01", #https://es.wikipedia.org/wiki/Protestas_en_Colombia_de_2021
                  max_id=max_id
                      ).items(max_items) #empty for all 

    tweets_copy = []
    for tweet in tweets:
        tweets_copy.append(tweet)

    print("Total Tweets fetched:", len(tweets_copy))
    max_id=tweets_copy[-1]._json.get('id')
    max_id=max_id-1

    step_df=pd.DataFrame()
    # populate the dataframe
    for tweet in tweets_copy:
        hashtags = []
        try:
            for hashtag in tweet.entities["hashtags"]:
                hashtags.append(hashtag["text"])
            text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
        except:
            pass

        original=None
        if tweet._json.get('retweeted_status'):
            original=tweet.retweeted_status.id
        step_df = step_df.append({'user_name': tweet.user.name, 
                                                   'user_location': tweet.user.location,\
                                                   'user_description': tweet.user.description,
                                                   'user_verified': tweet.user.verified,
                                                   'date': tweet.created_at,
                                                   'text': text, 
                                                   'hashtags': [hashtags if hashtags else None],
                                                   'source': tweet.source,
                                                   'id':tweet.id,
                                                   'original_id':original},
                                 ignore_index=True)
        step_df = step_df.reset_index(drop=True)
    tweets_df=tweets_df.append(step_df).reset_index(drop=True)
    # show the dataframe
    print('')

Total Tweets fetched: 5000

Total Tweets fetched: 5000

Total Tweets fetched: 5000

Total Tweets fetched: 5000



In [39]:
tweets_df['id'].unique().shape

(19999,)

In [41]:
tweets_df=tweets_df.append( pd.read_json('tweets_df.json')).reset_index(drop=True)
tweets_df.shape

(25000, 10)

### Ejemplo de análisis de datos
Obtenga la lista de hashtags

In [26]:
hstgs=tweets_df["hashtags"].dropna().sum()
str(
    #first five elements
    hstgs[:5]
).replace("]",",...]")

"['Movilización', 'Cali', 'ParoNacional', 'ParoNacional', 'Cali',...]"

In [27]:
len(hstgs)

6271

veces

Obtener lista de hashtags únicos

In [29]:
str(
 #first five elements
 list(set(hstgs))[:5]
).replace(']',',...]')

"['DesconciertoNacional', 'Rechazo', 'AfroSoul', 'Ansiedad', 'JesúsSantrich',...]"

TOTAL:

In [30]:
len(set(hstgs))

604

__Actividad__: Organizar los hashtags en orden decendiente de acuerdo al número de veces que aparecen en la lista completa de los mismos

Hint:

In [ ]:
l=[2,8,7,8,6,7,8]

In [ ]:
l.count(8)

In [35]:
hs=pd.DataFrame()

In [42]:
pd.DataFrame( zip( set(hstgs) , [ hstgs.count(x) for x in set(hstgs)] ),columns=['hastags','count'] ).sort_values('count',ascending=False)[:50]

,hastags,count
323,ParoNacional,1824
381,Colombia,420
67,CALI,382
452,Cali,293
156,ParoNacional18M,183
548,18M,101
441,PrimeraLínea,99
504,NoMasParo,91
278,Yumbo,77
261,Bogotá,64


### Grabar el objeto para análisis posteriores

In [48]:
import pickle
f=open('tweets_copy.pickle','rb')
tw_cp=pickle.load(f)
f.close()

In [95]:
tweet=tw_cp[9]

In [96]:
original=None
if tweet._json.get('retweeted_status'):
    original=tweet.retweeted_status.id
original

# Full DATA:
El atributo `._json` nos permite acceder a la información no estructurado del objeto de tweet completo desglosado en: https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet


In [165]:
print(f'{str(tweet._json)[:76]}...')

{'created_at': 'Wed May 12 05:52:23 +0000 2021', 'id': 1392356965578420225, ...


Si se adiciona como la fila de un DataFrame podremos visuzalizar la base de datos no estructurada completa

In [76]:
full_df=pd.DataFrame()
for tweet in tweets_copy:
    full_df=full_df.append(tweet._json,ignore_index=True)

In [105]:
pd.set_option('display.max_columns',50)

In [107]:
full_df[:1]

,contributors,coordinates,created_at,entities,favorite_count,favorited,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,metadata,place,retweet_count,retweeted,retweeted_status,source,text,truncated,user,extended_entities,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str
0,None,None,Wed May 12 05:52:23 +0000 2021,"{'hashtags': [], 'symbols': [], 'user_mentions...",0.0,0.0,None,1392356965578420224,1392356965578420225,None,None,None,None,None,0.0,es,"{'iso_language_code': 'es', 'result_type': 're...",None,77.0,0.0,{'created_at': 'Wed May 12 01:10:27 +0000 2021...,"<a href=""http://twitter.com/download/android"" ...",RT @SemanarioVOZ: El Semanario VOZ recalca su ...,0.0,"{'id': 1392150058087428102, 'id_str': '1392150...",NaN,NaN,NaN,NaN,NaN


In [92]:
full_df['id']=full_df['id'].astype(int)

In [95]:
full_df.to_json('full_df.json',orient='records')

In [78]:
full_df=pd.read_json('full_df.json')

In [81]:
full_df['retweeted_status']

0    {'created_at': 'Wed May 12 01:10:27 +0000 2021...
1    {'created_at': 'Tue May 11 12:28:44 +0000 2021...
2    {'created_at': 'Sat May 08 00:53:44 +0000 2021...
3    {'created_at': 'Wed May 12 03:59:21 +0000 2021...
4    {'created_at': 'Wed May 12 00:48:13 +0000 2021...
5    {'created_at': 'Tue May 11 22:12:00 +0000 2021...
6                                                 None
7                                                 None
8    {'created_at': 'Wed May 12 05:25:58 +0000 2021...
9    {'created_at': 'Tue May 11 13:15:47 +0000 2021...
Name: retweeted_status, dtype: object

_________